In [1]:
import scipy.misc as sm
from scipy import ndimage
import numpy as np
import matplotlib.pyplot as plt
from skimage import data
from skimage import filters
from skimage.morphology import erosion, disk

# a) Read the grayscale image brain.png, which is provided on the lecture homepage. Reduce the salt and pepper noise in the image using a median filter.

In [2]:
png_file = sm.imread("brain.png", flatten = False, mode = 'L')
plt.imshow(png_file)
plt.gray()
plt.title('Main image')
plt.show()

med_denoised = ndimage.median_filter(png_file, 5)
plt.title('Denoised image')
plt.imshow(med_denoised)
plt.show()

# b) Otsu thresholding is a histogram based method for image segmentation. Use it to find an intensitythreshold to segment brain pixels from background. Use Otsu thresholding again to find the threshold only over the brain pixels to segment brain's gray matter from the white matter. Using the two thresholds create three binary masks brain-bg.png, brain-gm.png, brain-wm.png, which should be white in regions of background, gray matter, and white matter, respectively, and black elsewhere.

In [3]:
val = filters.threshold_otsu(png_file)
#val = filters.threshold_otsu(med_denoised)
mask = med_denoised > val
print('background separating threshold', val)
plt.title('mask for background')
plt.imshow(mask)
plt.show()

brain_matter = np.multiply(png_file, mask)
#brain_matter = np.multiply(med_denoised, mask)
plt.title('Background substracted image')
plt.imshow(brain_matter)
plt.show()

#Converting the background 1 to use the same filter again.. If we dont change the value then then the filter cannot be used..Everytime it will
#separate from background
background = (brain_matter == 0)
brain_matter[background] = 255
val1 = filters.threshold_otsu(brain_matter)
print('gray matter from white matter separating threshold', val1)
#mask1 = brain_matter > val1
#brain_matter0 = np.multiply(brain_matter, mask1)

print('Thresholds:', val, val1)

#Binary mask for background
#deep copy, the values are copied; in any case if we change the values after denoising
brain_bg = np.copy(png_file)
#brain_bg = np.copy(med_denoised)
brain_bg = brain_bg < val
sm.imsave('C:\\Users\\Tamal\\Documents\\3rd Semester\\Bioinfo ll\\Last Assignment Tasks\\Task 6\\task_b_brain-bg.png',brain_bg)

#Binary mask for grey matter
brain_gm = np.copy(png_file)
#brain_gm = np.copy(med_denoised)
brain_gm = np.logical_and(brain_gm>=val, brain_gm < val1)
sm.imsave('C:\\Users\\Tamal\\Documents\\3rd Semester\\Bioinfo ll\\Last Assignment Tasks\\Task 6\\task_b_brain-gm.png',brain_gm)

#Binary mask for white matter
brain_wm = np.copy(png_file)
#brain_wm = np.copy(med_denoised)
brain_wm = brain_wm >= val1
sm.imsave('C:\\Users\\Tamal\\Documents\\3rd Semester\\Bioinfo ll\\Last Assignment Tasks\\Task 6\\task_b_brain-wm.png',brain_wm)

background separating threshold 106
gray matter from white matter separating threshold 206
Thresholds: 106 206


# c) Plot a log-scaled histogram of the image, which should show how frequently different intensity values occur in the image. Mark the two intensity thresholds you found in task b). How do thesethresholds divide the histogram?

In [4]:
plt.hist(png_file.ravel())
#plt.hist(med_denoised.ravel())
plt.title('Log-scaled histogram of different intensity values')
plt.xlabel('Intensity')
plt.ylabel('Frequency')
plt.yscale('log')
plt.axvline(val, color='r', linestyle='dashed', linewidth=2)
plt.axvline(val1, color='r', linestyle='dashed', linewidth=2)
plt.savefig('C:\\Users\\Tamal\\Documents\\3rd Semester\\Bioinfo ll\\Last Assignment Tasks\\Task 6\\task_c_Log_scaled_Histogram.png')
plt.show()

# d) Combine the three masks into a single color image so that background, gray matter, and white matter are mapped to red, green and blue, respectively.

In [5]:
brain_rgb = np.dstack((brain_bg,brain_gm,brain_wm))
plt.imshow(brain_rgb)
plt.savefig('C:\\Users\\Tamal\\Documents\\3rd Semester\\Bioinfo ll\\Last Assignment Tasks\\Task 6\\task_d_brain_rgb.png')
plt.show()

# e) Use erosion filter to produce a boundary between the gray and white matter. Put that boundary on the denoised input image.

In [6]:
brain_wm1 = np.copy(med_denoised)
brain_wm1 = brain_wm1 >= val1
white_matter = np.multiply(med_denoised, brain_wm1)
#plt.imshow(white_matter)
#plt.show()

selem = disk(1)
eroded = erosion(white_matter, selem)
#plt.imshow(eroded)
#plt.show()

inverted_wh_mask = 1 - brain_wm1
#plt.imshow(inverted_wh_mask)
#plt.show()

brain_contour = np.multiply(med_denoised, inverted_wh_mask) + eroded
plt.imshow(brain_contour)
plt.savefig('C:\\Users\\Tamal\\Documents\\3rd Semester\\Bioinfo ll\\Last Assignment Tasks\\Task 6\\task_e_brain_contour.png')
plt.show()

# f) Use bilinear interpolation to upsample the image by a factor of four along each axis. Apply the same thresholds as in b) to obtain a segmentation into background, gray matter, and white matter. Upsample the masks from b) in the same way and compare the upsampled masks to the masks from the upsampled image. Can you see a difference? Why? (4P)

In [7]:
brain_fig = np.copy(png_file)
upsampled_brain_fig = ndimage.zoom(brain_fig, 4, order=1)
sm.imsave('C:\\Users\\Tamal\\Documents\\3rd Semester\\Bioinfo ll\\Last Assignment Tasks\\Task 6\\task_f_Upsampled_brain_fig.png',upsampled_brain_fig)

#plt.imshow(upsampled_brain)
#plt.show()
upsampled_bg = upsampled_brain_fig
upsampled_bg = upsampled_bg < val
plt.imshow(upsampled_bg,cmap = 'gray', vmin = 0 ,vmax = 1)
plt.show()
sm.imsave('C:\\Users\\Tamal\\Documents\\3rd Semester\\Bioinfo ll\\Last Assignment Tasks\\Task 6\\task_f_Upsampled_bg.png',upsampled_bg)

upsampled_wm = upsampled_brain_fig
upsampled_wm = upsampled_wm >= val1
plt.imshow(upsampled_wm,cmap ='gray',vmin = 0,vmax = 1)
plt.show()
sm.imsave('C:\\Users\\Tamal\\Documents\\3rd Semester\\Bioinfo ll\\Last Assignment Tasks\\Task 6\\task_f_Upsampled_wm.png',upsampled_wm)

upsampled_gm = upsampled_brain_fig
upsampled_gm = np.logical_and(upsampled_gm >= val, upsampled_gm < val1)
plt.imshow(upsampled_gm,cmap = 'gray',vmin = 0,vmax = 1)
plt.show()
sm.imsave('C:\\Users\\Tamal\\Documents\\3rd Semester\\Bioinfo ll\\Last Assignment Tasks\\Task 6\\task_f_Upsampled_gm.png',upsampled_gm)

upsampled_brain_bg=ndimage.zoom(brain_bg + 0, 4, order=1)
plt.imshow(upsampled_brain_bg,cmap='gray',vmin=0,vmax=1)
plt.show()
sm.imsave('C:\\Users\\Tamal\\Documents\\3rd Semester\\Bioinfo ll\\Last Assignment Tasks\\Task 6\\task_f_Upsampled_brain_bg.png',upsampled_brain_bg)

upsampled_brain_wm = ndimage.zoom(brain_wm+0, 4, order = 1)
plt.imshow(upsampled_brain_wm,cmap ='gray',vmin = 0,vmax = 1)
plt.show()
sm.imsave('C:\\Users\\Tamal\\Documents\\3rd Semester\\Bioinfo ll\\Last Assignment Tasks\\Task 6\\task_f_Upsampled_brain_wm.png',upsampled_brain_wm)

upsampled_brain_gm = ndimage.zoom(brain_gm+0, 4, order = 1)
plt.imshow(upsampled_brain_gm,cmap = 'gray',vmin = 0,vmax = 1)
plt.show()
sm.imsave('C:\\Users\\Tamal\\Documents\\3rd Semester\\Bioinfo ll\\Last Assignment Tasks\\Task 6\\task_f_Upsampled_brain_gm.png',upsampled_brain_gm)

brain_fig_NNI = np.copy(png_file)
NNI_brain = ndimage.zoom(brain_fig_NNI, 4, order = 0)
#plt.imshow(NNI_brain)
#plt.show()

NNI_brain_bg = NNI_brain
NNI_brain_bg = NNI_brain_bg < val
plt.imshow(NNI_brain_bg,cmap ='gray',vmin = 0,vmax = 1)
plt.show()
sm.imsave('C:\\Users\\Tamal\\Documents\\3rd Semester\\Bioinfo ll\\Last Assignment Tasks\\Task 6\\task_f_NNI_brain_bg.png',NNI_brain_bg)

NNI_brain_wm = NNI_brain
NNI_brain_wm = NNI_brain_wm >= val1
plt.imshow(NNI_brain_wm,cmap ='gray',vmin = 0,vmax = 1)
plt.show()
sm.imsave('C:\\Users\\Tamal\\Documents\\3rd Semester\\Bioinfo ll\\Last Assignment Tasks\\Task 6\\task_f_NNI_brain_wm.png',NNI_brain_wm)

NNI_brain_gm = NNI_brain
NNI_brain_gm = np.logical_and(NNI_brain_gm >= val, NNI_brain_gm < val1)
plt.imshow(NNI_brain_gm,cmap ='gray',vmin = 0,vmax = 1)
plt.show()
sm.imsave('C:\\Users\\Tamal\\Documents\\3rd Semester\\Bioinfo ll\\Last Assignment Tasks\\Task 6\\task_f_NNI_brain_gm.png',NNI_brain_gm)

#Upsampling the masks from part b
NNI_bg_mask = ndimage.zoom(brain_bg, 4, order = 0)
plt.imshow(NNI_bg_mask,cmap ='gray',vmin = 0,vmax = 1)
plt.show()
sm.imsave('C:\\Users\\Tamal\\Documents\\3rd Semester\\Bioinfo ll\\Last Assignment Tasks\\Task 6\\task_f_NNI_bg_mask.png',NNI_bg_mask)

NNI_wm_mask = ndimage.zoom(brain_wm, 4, order = 0)
plt.imshow(NNI_wm_mask,cmap ='gray',vmin = 0,vmax = 1)
plt.show()
sm.imsave('C:\\Users\\Tamal\\Documents\\3rd Semester\\Bioinfo ll\\Last Assignment Tasks\\Task 6\\task_f_NNI_wm_mask.png',NNI_wm_mask)

NNI_gm_mask = ndimage.zoom(brain_gm, 4, order = 0)
plt.imshow(NNI_gm_mask,cmap ='gray',vmin = 0,vmax = 1)
plt.show()
sm.imsave('C:\\Users\\Tamal\\Documents\\3rd Semester\\Bioinfo ll\\Last Assignment Tasks\\Task 6\\task_f_NNI_gm_mask.png',NNI_gm_mask)



